In [41]:
library(data.table)
library(lmtest)
library(broom)

In [53]:
n <- 1000
d <- data.table(x=sample(18:39, n, replace=T), edu=sample(1:3, n, replace=T), treat=sample(0:1, n, replace=T))
d$edu <- as.factor(d$edu)
d <- d[, y:=x*1000 + treat * 5000 + rnorm(n, mean=0, sd=100)]
d <- d[, y:=y+(edu==1)*50*treat]
d <- d[, y:=y+(edu==2)*250*treat]
d <- d[, y:=y+(edu==3)*1000*treat]

In [54]:
m <- lm(y ~ x + treat + treat:edu, data = d)
#summary(m)

In [55]:
coefs <- data.table(tidy(coeftest(m)))
coefs

,term,estimate,std.error,statistic,p.value
1,(Intercept),-10.82363,15.39669,-0.7029841,0.4822301
2,x,1000.393,0.5106388,1959.101,0
3,treat,5048.048,9.202203,548.5696,0
4,treat:edu2,195.4978,11.24626,17.38336,2.612166e-59
5,treat:edu3,965.1649,11.00117,87.73291,0


In [56]:
# Effect of 'treat' when edu == 1
# Should be 5000 + 50
baseline <- coefs[3,estimate]
baseline

[1] 5048.048

In [57]:
# Effect of 'treat' when edu == 2
# Should be 5000 + 250

# the treat:edu2 coefficients means the effect of treat+edu2 when edu is changed from 1 to 2.
coefs[4,estimate] + baseline 

[1] 5243.546

In [58]:
# Effect of 'treat' when edu == 3
# Should be 5000 + 1000
coefs[5,estimate] + baseline 

[1] 6013.213